## Problem 1: Generating Random Boolean Functions

### Background

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) is one of the earliest quantum algorithms, developed by David Deutsch and Richard Jozsa in 1992. It was designed to demonstrate a clear separation between quantum and classical computation, showing that quantum computers can solve certain problems exponentially faster than classical computers.

The algorithm works with **Boolean functions** — functions that accept a fixed number of Boolean inputs (each either `True` or `False`) and return a single Boolean output. For a function with $n$ inputs, there are $2^n$ possible input combinations.

The key constraint is that each function must be one of two types:
- **Constant**: The function returns the same value (`True` or `False`) for *all* possible input combinations
- **Balanced**: The function returns `True` for *exactly half* of the input combinations and `False` for the other half

This problem focuses on generating random functions that satisfy these constraints, which will later be used to compare classical and quantum approaches to determining the function type.

### Solution Explanation

The idea for this problem is to randomly generate either a constant value (`True` or `False`) for all possible combinations of n binary inputs, or return True (which are randomly distributed) for exactly half of the combinations (and False for the other half).

For n inputs there will be $2^n$ outputs as there are $2^n$ combinations.

In [2]:
# Required imports
import random
import numpy as np
from itertools import product

In [3]:
def random_constant_balanced(num_inputs):
    """
    Generate a random constant or balanced truth table for a given number of inputs.
    Args:
        num_inputs (int): Number of input variables.
    Returns:
        np.ndarray: An array representing the truth table of length 2**num_inputs, either constant or balanced.
    """

    table_size = 2 ** num_inputs # This will always be even, so we do not have to worry about uneven True and False counts.
    
    # Randomly choose between 'constant' and 'balanced'
    if random.choice(['constant', 'balanced']) == 'constant':
        return np.full(table_size, random.choice(['0', '1']))
    else:
        half = table_size // 2
        result = np.array(['0'] * half + ['1'] * half)
        np.random.shuffle(result)
        return result

In [4]:
def test_random_constant_balanced():
    for num_inputs in range(1, 21):  # Testing for 1 to 20 inputs
        for _ in range(10):  # Run multiple tests for each input size
            table = random_constant_balanced(num_inputs)
            assert len(table) == 2 ** num_inputs, f"Failed length test for {num_inputs} inputs"
            zeros = np.sum(table == '0')
            ones = np.sum(table == '1')
            assert zeros + ones == len(table), f"Failed value test for {num_inputs} inputs"
            assert zeros == ones or zeros == 0 or ones == 0, f"Failed balance/constant test for {num_inputs} inputs"
    print("All tests passed.")

In [5]:
test_random_constant_balanced()

All tests passed.


## Problem 2: Classical Testing for Function Type

### Background

[Deutsch's algorithm](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm), developed by David Deutsch in 1985, was the first quantum algorithm to demonstrate a [potential advantage of quantum computing](https://www.quantamagazine.org/john-preskill-explains-quantum-supremacy-20191002/) over classical computation. It addresses a question: given a black-box function (oracle), can we determine a global property of that function more efficiently using quantum mechanics?

The problem is framed as follows: we are given a function $f: \{0, 1\}^n \rightarrow \{0, 1\}$ that is promised to be either **constant** or **balanced**. Our task is to determine which type it is by querying (calling) the function as few times as possible.

**Classical Limitation**: Classically, in the worst case, we must query the function at least $2^{n-1} + 1$ times to be certain of the answer. This is because a balanced function could match a constant function's output for up to half of all inputs before revealing a different value.

**Quantum Advantage**: Deutsch's algorithm (and its generalisation, the Deutsch-Jozsa algorithm) can determine the function type with just **one** query, regardless of the number of inputs. This exponential speedup, while on a contrived problem, was historically significant as proof that quantum computers could outperform classical ones.

Before exploring the quantum solution, we must first understand the classical approach and its limitations.

### Solution Explanation

This problem requires writing a function called `determine_constant_balanced`, which takes a parameter `f` (the function written in Problem 1). `f` is the truth table returned from the function.

`determine_constant_balanced` returns whether the function is `constant` or `balanced` based on the black box output.

Since constant can either return True or False for every index in the truth table, we can take the initial value and save it in a variable `first_value`. We then loop through each index and if any index does not equal `first_value`, then the function must be `balanced`. If the first `len(f) // 2 + 1` indices of the truth table `f` all equal `first_value`, then the function is guaranteed to be constant, as more than half of the truth table is equal to `first_value`. Note that `len(f) // 2 + 1` can be expressed as $2^{n-1} + 1$ where $n$ is the `num_inputs`.

In [6]:
def determine_constant_balanced(f):
    """
    f is the truth table array from Problem 1.
    Returns 'constant' or 'balanced'.
    """

    # First value to compare to other indices.
    first_value = f[0]
    
    # Check only half the table + 1 since balanced means equal counts.
    for i in range(1, len(f) // 2 + 1):
        if f[i] != first_value:
            return 'balanced'  # Found a different value -> method must be balanced.
    
    return 'constant'  # All values were the same -> method must be constant.

### Efficiency
`determine_constant_balanced` has an `O(n)` runtime in relation to `num_inputs` and an `O(n)` runtime in relation to the size of the array input.

#### Best and Worst Case Scenarios
| Scenario | Number of calls to `f` | Explanation |
|----------|----------------------|-------------|
| **Best case** | 2 | Find two different outputs immediately → balanced |
| **Worst case** | 2<sup>n-1</sup> + 1 | Must check just over half the table |

`2ⁿ⁻¹ + 1` is the worst case as we can guarantee that the function is constant if over half of the truth table is the same value.

### Testing

In [7]:
def test_determine_constant_balanced():
    """
    Test cases for determine_constant_balanced function.
    Tests both constant and balanced functions with various input sizes.
    """
    
    # Test 1: Constant - size 2 (n=1)
    small_constant = np.array(['0', '0'])
    assert determine_constant_balanced(small_constant) == 'constant', "Failed: small constant"
    print("Test 1 passed.")
    
    # Test 2: Balanced - size 2 (n=1)
    small_balanced = np.array(['0', '1'])
    assert determine_constant_balanced(small_balanced) == 'balanced', "Failed: small balanced"
    print("Test 2 passed.")
    
    # Test 3: Constant - size 8 (n=3)
    constant_medium = np.array(['1'] * 8)
    assert determine_constant_balanced(constant_medium) == 'constant', "Failed: medium constant"
    print("Test 3 passed.")
    
    # Test 4: Balanced - size 8 with early detection (n=3)
    # 4 zeros and 4 ones, difference at position 1
    balanced_medium = np.array(['0', '1', '0', '1', '0', '1', '0', '1'])
    assert determine_constant_balanced(balanced_medium) == 'balanced', "Failed: medium balanced"
    print("Test 4 passed.")
    
    # Test 5: Constant - size 16 (n=4)
    constant_large = np.array(['0'] * 16)
    assert determine_constant_balanced(constant_large) == 'constant', "Failed: large constant"
    print("Test 5 passed.")
    
    # Test 6: Balanced - size 16 (n=4)
    balanced_large = np.array(['0'] * 8 + ['1'] * 8)
    assert determine_constant_balanced(balanced_large) == 'balanced', "Failed: large balanced"
    print("Test 6 passed.")
    
    # Test 7: Constant - size 1024 (n=10)
    large_constant = np.array(['1'] * 1024)
    assert determine_constant_balanced(large_constant) == 'constant', "Failed: very large constant"
    print("Test 7 passed.")
    
    # Test 8: Balanced - size 1024 (n=10)
    large_balanced = np.array(['0'] * 512 + ['1'] * 512)
    assert determine_constant_balanced(large_balanced) == 'balanced', "Failed: very large balanced"
    print("Test 8 passed.")
    
    # Test 9: Test with Problem 1 function - size 2048 (n=11)
    for i in range(1, 21):  # Run 20 tests
        var = determine_constant_balanced(random_constant_balanced(11))
        assert var in ['constant', 'balanced'], "Failed: random function of size 2048"
        print(f"Test 9: Function subtest {i}: {var} - passed.")

    print("All test cases passed.")

In [8]:
# Run the test
test_determine_constant_balanced()

Test 1 passed.
Test 2 passed.
Test 3 passed.
Test 4 passed.
Test 5 passed.
Test 6 passed.
Test 7 passed.
Test 8 passed.
Test 9: Function subtest 1: constant - passed.
Test 9: Function subtest 2: constant - passed.
Test 9: Function subtest 3: constant - passed.
Test 9: Function subtest 4: balanced - passed.
Test 9: Function subtest 5: constant - passed.
Test 9: Function subtest 6: constant - passed.
Test 9: Function subtest 7: balanced - passed.
Test 9: Function subtest 8: balanced - passed.
Test 9: Function subtest 9: balanced - passed.
Test 9: Function subtest 10: constant - passed.
Test 9: Function subtest 11: constant - passed.
Test 9: Function subtest 12: constant - passed.
Test 9: Function subtest 13: balanced - passed.
Test 9: Function subtest 14: constant - passed.
Test 9: Function subtest 15: constant - passed.
Test 9: Function subtest 16: balanced - passed.
Test 9: Function subtest 17: constant - passed.
Test 9: Function subtest 18: balanced - passed.
Test 9: Function subtest 

## Problem 3: Quantum Oracles

### Background

[Deutsch's algorithm](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm) is the simplest example of a [quantum algorithm](https://www.ibm.com/quantum/blog/group-theory) that uses [superposition](https://scienceexchange.caltech.edu/topics/quantum-science-explained/quantum-superposition) to determine a [global property](https://plato.stanford.edu/archives/fall2008/entries/qt-entangle/#5) of a function with a single evaluation.

In Problem 2, we explored how many calls to the `random_constant_balanced` function would be required to confirm that the function is `constant` or `balanced` using Classical Computing. In the worst-case scenario, we require $2^{n-1} + 1$ calls to `random_constant_balanced`. Deutsch's algorithm only requires 1 call, which is exponentially faster, and represents the potential advantages of Quantum Computing over Classical Computing.

#### The Four Single-Input Boolean Functions

For a function $f: \{0, 1\} \rightarrow \{0, 1\}$ with a single Boolean input, there are exactly four possible functions:

| Function | $f(0)$ | $f(1)$ | Type |
|----------|--------|--------|------|
| $f_0$ | 0 | 0 | Constant |
| $f_1$ | 1 | 1 | Constant |
| $f_2$ | 0 | 1 | Balanced (Identity) |
| $f_3$ | 1 | 0 | Balanced (NOT) |

#### What is a Quantum Oracle?

A [quantum oracle](https://quantumcomputing.stackexchange.com/a/4626) is a "black box" operation that encodes a classical function into a quantum circuit. Unlike classical function evaluation, a quantum oracle must be reversible (all quantum operations are unitary and thus reversible).

#### What is an Ancilla Qubit?

An [ancilla qubit](https://www.sciencedirect.com/topics/mathematics/ancilla-qubit) is a "helper" qubit that, unlike information qubits, starts in a known state and is used to enable or simplify complex algorithms. In this function they make the output state reversible to input state.

The standard way to implement an oracle for a function $f(x)$ is using the **phase kickback** technique with an ancilla qubit:

$$U_f |x\rangle |y\rangle = |x\rangle |y \oplus f(x)\rangle$$

Where $\oplus$ denotes XOR (addition modulo 2). This transformation:
- Leaves the input qubit $|x\rangle$ unchanged
- Flips the ancilla qubit $|y\rangle$ if and only if $f(x) = 1$

#### Why does this use the $\oplus$ (XOR) operation?

This algorithm uses an interesting property of the $\oplus$ operator. For any values `a` and `b`:
- $a \oplus b = b \oplus a$ (XOR is the addition 2 operation and thus is commutative)
- $b \oplus b = 0$ (XORs cancel out)
- $a \oplus b \oplus b = a$ (XORs cancel out and we are left with $a$)


This construction is reversible because applying the same operation twice returns the original state.

### Solution Explanation

To implement the four oracles, we need to understand how each function translates to quantum gates:

| Function | Behaviour | Oracle Implementation |
|----------|-----------|----------------------|
| $f_0(x) = 0$ | Always returns 0 | Do nothing (identity) — the ancilla is never flipped |
| $f_1(x) = 1$ | Always returns 1 | Apply X gate to ancilla — always flip it |
| $f_2(x) = x$ | Returns the input (identity) | Apply CNOT with input as control — flip ancilla when input is 1 |
| $f_3(x) = \neg x$ | Returns NOT of input | Apply CNOT then X to ancilla — flip ancilla when input is 0 |

Each oracle is implemented as a function that takes a `QuantumCircuit` and applies the appropriate gates to encode the function $f$.

### Solution

In [9]:
# Required imports
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt